In [1]:
using IntervalConstraintProgramming, ValidatedNumerics

  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31


In [3]:
disc = @constraint x^2+y^2 <= 1

code = quote  # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 18: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 14:
    z4 = x ^ 2 # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 18: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 14:
    z5 = y ^ 2
    z6 = z4 + z5
    z6 = z6 ∩ _A_
    (z6,z4,z5) = plus_rev(z6,z4,z5)
    (z5,y,_) = power_rev(z5,y,2)
    (z4,x,_) = power_rev(z4,x,2)
    return (x,y)
end


Separator with variables x, y

In [4]:
solid_torus = @constraint (3 - sqrt(x^2 + y^2))^2 + z^2 <= 1

code = quote  # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 14: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 18: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 14:
    z7 = x ^ 2 # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, line 38: # /Users/dpsanders/.julia/v0.5/IntervalConstraintProgramming/src/contractor.jl, l

Separator with variables x, y, z

In [5]:
Y = IntervalBox(-5..5, -5..5, -5..5)
@time paving = setinverse(solid_torus, Y, 0.25);

  3.712546 seconds (10.17 M allocations: 381.060 MB, 5.05% gc time)


In [6]:
length(paving.inner)

3520

  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/dpsanders/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /Users/dpsanders/.julia/v0.5/IJulia/src/execute_request.jl


## 3D vis 

In [ ]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors

In [ ]:
inner = paving.inner
positions = Point3{Float32}[Point3(mid(x)) for x in inner]
scales = Vec3f0[Vec3f0([diam(x) for x in xx]) for xx in inner]
colors = [RGBA(1f0, 0f0, 0f0, 0.1f0) for xx in inner];

In [7]:
using GLVisualize, GeometryTypes, GLAbstraction, Colors
println("Visualization packages loaded.")

cube_particle = HyperRectangle{3, Float32}(Vec3f0(-0.5, -0.5, -0.5), Vec3f0(1, 1, 1))

window = glscreen()

vis = visualize( (cube_particle, positions), scale=scales, color=colors)

view(vis)


renderloop(window)

INFO: Precompiling module GLVisualize...
  likely near /Users/dpsanders/.julia/v0.4/GLVisualize/src/FreeTypeAbstraction.jl:18
  likely near /Users/dpsanders/.julia/v0.4/GLVisualize/src/FreeTypeAbstraction.jl:18
ERROR: LoadError: LoadError: syntax: invalid method name "::Type{AABB}"
 in include at /Applications/Julia-0.4.6.app/Contents/Resources/julia/lib/julia/sys.dylib
 in include_from_node1 at /Applications/Julia-0.4.6.app/Contents/Resources/julia/lib/julia/sys.dylib
 in include at /Applications/Julia-0.4.6.app/Contents/Resources/julia/lib/julia/sys.dylib
 in include_from_node1 at /Applications/Julia-0.4.6.app/Contents/Resources/julia/lib/julia/sys.dylib
 [inlined code] from none:2
 in anonymous at no file:0
 in process_options at /Applications/Julia-0.4.6.app/Contents/Resources/julia/lib/julia/sys.dylib
 in _start at /Applications/Julia-0.4.6.app/Contents/Resources/julia/lib/julia/sys.dylib
while loading /Users/dpsanders/.julia/v0.4/GLVisualize/src/utils.jl, in expression starting o

LoadError: LoadError: Failed to precompile GLVisualize to /Users/dpsanders/.julia/lib/v0.4/GLVisualize.ji
while loading In[7], in expression starting on line 1